# LSTM Model

In [1]:
import numpy as np
import pickle
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import TimeDistributed, Dense, LSTM, Layer
from keras import Input

ModuleNotFoundError: No module named 'tensorflow'

In [2]:
with open('HCP_movie_watching.pkl','rb') as f:
    TS = pickle.load(f)

%store -r testIndex

In [3]:
def splitDict(dict, indexList = testIndex):
    trainDict = {}
    testDict = {}
    for key, val in dict.items():
        testvalArr = []
        trainvalArr = []
        if key == "testretest":
            testvalArr = testvalArr
            for i in range(val.shape[0]):
                for p in range(val.shape[1]):
                    if p in testIndex:
                        testvalArr.append(val[i][p])
                    else:
                        trainvalArr.append(val[i][p])
        else:
            for p in range(val.shape[0]):
                if p in testIndex:
                        testvalArr.append(val[p])
                else:
                    trainvalArr.append(val[p])
        trainDict[key] = np.array(trainvalArr)
        testDict[key] = np.array(testvalArr)
    return trainDict, testDict


In [4]:
def vectorize_labels(labels, keys, classNums = 13):
    results = np.zeros((int(labels.shape[0]), int(labels.shape[1]), classNums))
    for i in range(labels.shape[0]):
        for j in range(labels.shape[1]):
            if labels[i][j] != -100.:
                indexNum = keys.index(labels[i][j])
                results[i][j][indexNum] = 1.
    return results


In [5]:
# Reshape data into 3D array

def shaping(dict):
    X_arr = []
    y_arr = []
    keylist = list(dict.keys())
    for key, val in dict.items():
        for i in range(val.shape[0]):
            normalized_seq = (val[i] - np.mean(val[i])) / np.std(val[i])
            X_arr.append(normalized_seq)
            clip = [key for j in range(min(val.shape[1], 90))]
            y_arr.append(clip)
        
    X_padded = tf.keras.preprocessing.sequence.pad_sequences(X_arr, maxlen = 90, dtype='float64', padding='post', truncating='post')
    y_padded = tf.keras.preprocessing.sequence.pad_sequences(y_arr, maxlen = 90, dtype = 'object', padding='post', truncating='post', value = -100.)
    y_padded = vectorize_labels(y_padded, keylist, classNums = len(keylist))

    return tf.convert_to_tensor(X_padded, dtype='float64'), tf.convert_to_tensor(y_padded)

In [6]:
train, test = splitDict(TS)
X_train, y_train = shaping(train)
X_test, y_test = shaping(test)

In [17]:
def createMask(X):
    sample = X.shape[0]
    time_seq = X.shape[1]

    mask = np.empty((sample, time_seq), dtype=np.bool_)
    for i in range(sample):
        for j in range(time_seq):
            if np.count_nonzero(X[i][j]) == 0:
                mask[i][j] = False
            else:
                mask[i][j] = True
    return tf.convert_to_tensor(mask)


In [18]:
mask = createMask(X_train)

In [21]:
inputs = Input(shape=(X_train.shape[1],X_train.shape[2]))
x = LSTM(units=32, activation='tanh', recurrent_activation='sigmoid', return_sequences=True, dropout=0.05)(inputs)
outputs = TimeDistributed(Dense(15, activation='softmax'))(x)

model = keras.Model(inputs, outputs)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [22]:
history = model.fit(X_train, y_train, batch_size = 32, epochs = 60, validation_split=0.2)

Epoch 1/60
45/45 [==============================] - 20s 133ms/step - loss: 2.3635 - accuracy: 0.2564 - val_loss: 3.6496 - val_accuracy: 6.1728e-04
Epoch 2/60
45/45 [==============================] - 3s 74ms/step - loss: 2.0578 - accuracy: 0.3539 - val_loss: 4.0409 - val_accuracy: 2.1605e-04
Epoch 3/60
45/45 [==============================] - 3s 70ms/step - loss: 1.7994 - accuracy: 0.4350 - val_loss: 4.2560 - val_accuracy: 0.0011
Epoch 4/60
45/45 [==============================] - 3s 64ms/step - loss: 1.5729 - accuracy: 0.5142 - val_loss: 4.7349 - val_accuracy: 0.0013
Epoch 5/60
45/45 [==============================] - 3s 69ms/step - loss: 1.3534 - accuracy: 0.5942 - val_loss: 5.3094 - val_accuracy: 0.0019
Epoch 6/60
45/45 [==============================] - 3s 68ms/step - loss: 1.1270 - accuracy: 0.6689 - val_loss: 5.8335 - val_accuracy: 0.0032
Epoch 7/60
45/45 [==============================] - 3s 64ms/step - loss: 0.9339 - accuracy: 0.7296 - val_loss: 6.3174 - val_accuracy: 0.0066
Epo

In [23]:
def findIndex(val, arr):
    index = -1
    for x in range(arr.size):
        if val == arr[x]:
            index = x
            break
    return index

In [24]:
def accuracy(target, prob):
    accArr = []
    for i in range(prob.shape[1]):
        correctCount = 0
        totalCount = 0
        for j in range(prob.shape[0]):
            if np.count_nonzero(target[j][i]) != 0:
                if findIndex(1., target[j][i]) == findIndex(np.amax(prob[j][i]), prob[j][i]):
                    correctCount += 1
                totalCount += 1
        a = correctCount / totalCount
        accArr.append(a)
    return accArr

In [25]:
pred_test = model.predict(X_test)
acc = accuracy(y_test.numpy(), pred_test)

43/43 [==============================] - 6s 52ms/step


In [26]:
%store acc

Stored 'acc' (list)
